<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# Описание

Нам необходимо создать для магазина инструмент, позволяющий разделять комментарии на позитивные и негативные, а затем отправлять негативные на модерацию. Для решения задачи мы будем использовать набор данных с разметкой о токсичности правок.

## Подготовка

1.1. Загрузим библиотеки, необходимые для решения нашей задачи и файл с размеченными данными.

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.pipeline import Pipeline

import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
import spacy
from tqdm import tqdm

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier 
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score

import time
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

import plotly.graph_objects as go

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 2.1 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1.2. Посмотрим и обработаем текст для выполнения нашей задачи.

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
display(data.head())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [4]:
#посмотрмим, есть ли пропуски
data.isna().sum()

text     0
toxic    0
dtype: int64

In [5]:
# посмотрим, есть ли дубликаты
data.duplicated().sum()

0

In [6]:
# посмотрим тип данных
data.dtypes

text     object
toxic     int64
dtype: object

1.3.Обработаем текст с помощью регулярных выражений.

In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [8]:
tqdm.pandas()
data['text'] = data['text'].progress_apply(clean_text)

100%|██████████| 159292/159292 [00:09<00:00, 16444.77it/s]


In [9]:
#лемматизация текста
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
def spacy_lemm(sentence):
    doc = nlp(sentence)
    return " ".join([token.lemma_ for token in doc])

data['text'] = data['text'].progress_apply(spacy_lemm)

100%|██████████| 159292/159292 [17:12<00:00, 154.22it/s]


# Вывод:

1. В нашем распоряжении 159291 строчки размеченных данных;
2. дубликатов и пропусков нет;
3. типы данных подходящие;
4. с помощью регулярных выражений мы обработали текст для дальнейшего изучения;
5. лемматизировали текст.

## Обучение

2.1. Подготовим признаки и целевой признак для обучения, выделим выборки.

In [10]:
X = data.drop('toxic', axis=1)
y = data['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=12345)

2.2.Обучим модели

А. LogisticRegression

In [11]:
%%time

lr_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=12345))])

params = {'clf__C': [0.1, 1, 10, 100],
          'clf__class_weight': ['balanced', None]}

lr_grid = GridSearchCV(estimator=lr_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
lr_grid.fit(X_train['text'], y_train)
lr_best_paramms = lr_grid.best_params_

print(lr_best_paramms)
print(lr_grid.best_score_)

{'clf__C': 10, 'clf__class_weight': 'balanced'}
0.7715114676784594
CPU times: user 14min 33s, sys: 9min 29s, total: 24min 3s
Wall time: 24min 5s


Б. LGBMClassifier

In [12]:
%%time

lgb_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LGBMClassifier(random_state=12345))])

params = {
  'clf__n_estimators': [200],
  'clf__learning_rate': [0.15, 0.25],
  'clf__max_depth': [8, 10, -1]}

lgb_grid = GridSearchCV(estimator=lgb_pipe, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
lgb_grid.fit(X_train['text'], y_train)
lgb_best_params = lgb_grid.best_params_

print(lgb_best_params)
print(lgb_grid.best_score_)

{'clf__learning_rate': 0.25, 'clf__max_depth': -1, 'clf__n_estimators': 200}
0.7706727368140607
CPU times: user 1h 25min 8s, sys: 27.9 s, total: 1h 25min 36s
Wall time: 1h 25min 53s


Подобрали параметры с помощью GridSearchCV с расчетом TF-IDF

In [13]:
vectorize = TfidfVectorizer(ngram_range=(1,3),
               min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)

In [14]:
X_train = vectorize.fit_transform(X_train['text'])
X_test = vectorize.transform(X_test['text'])

In [15]:
%%time
lr_m = LogisticRegression(C=10, class_weight='balanced', random_state=12345)
lr_m.fit(X_train, y_train)

CPU times: user 34.7 s, sys: 33.9 s, total: 1min 8s
Wall time: 1min 8s


LogisticRegression(C=10, class_weight='balanced', random_state=12345)

In [16]:
%%time
lgb_m = LGBMClassifier(learning_rate=0.15, max_depth=-1, n_estimators=200, random_state = 12345)
lgb_m.fit(X_train, y_train)

CPU times: user 8min 25s, sys: 2.29 s, total: 8min 27s
Wall time: 8min 31s


LGBMClassifier(learning_rate=0.15, n_estimators=200, random_state=12345)

2.3. Посчитаем F! нащих моделей на тестовой выборке для проверки эффективности и выбора лучшей модели

In [17]:
def scoring(fitted_model):
    test_pred = fitted_model.predict(X_test)
    test_f1 = f1_score(y_test, test_pred)
    
    print('F1 on test: {:.3f}'.format(test_f1))

In [18]:
scoring(lgb_m)

F1 on test: 0.777


In [19]:
scoring(lr_m)

F1 on test: 0.788


# Вывод:

мы обучили модели LGBMClassifier и LogisticRegression, посчитали F! нащих моделей на тестовой выборке

## Выводы

Для разработки  инструмента, позволяющего разделять комментарии на позитивные и негативные, а затем отправлять негативные на модерацию, мы рассмотрели и обработали данные с разметкой о токсичности правок.
Затем из этих данных сделали выборки для обучения моделей, обучили модели LGBMClassifier и LogisticRegression, посчитали F! нащих моделей на тестовой выборке.

F1 LGBMClassifier: 0.777
F1 LogisticRegression: 0.788

Обе модели соответствуют условию задачи (F1 не меньше 0.75), но F1 LogisticRegression больше, при этом время обучения модели значительно меньше.
Соответственно на основании проведенных вычислений предложим заказчику лучшую модель, а именно: LogisticRegression.